<a href="https://colab.research.google.com/github/Neel7317/Generative-Models/blob/main/QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig
from torch.quantization import quantize_dynamic

# Sample Dataset
data = [
    {"text": "The soccer team clinched a thrilling victory in overtime.", "label": 0},
    {"text": "The cricket championship finals attracted global attention.", "label": 0},
    {"text": "A new record was set in the 100-meter sprint.", "label": 0},
    {"text": "Tennis legends face off in a historic grand slam final.", "label": 0},
    {"text": "The basketball league announces the upcoming season schedule.", "label": 0},
    {"text": "Fans cheer as their favorite football club wins the title.", "label": 0},
    {"text": "Olympic gold medalist announces retirement from the sport.", "label": 0},
    {"text": "The marathon saw record participation this year.", "label": 0},
    {"text": "Baseball's star player hits a remarkable home run.", "label": 0},
    {"text": "The Formula 1 race witnessed a dramatic last-lap overtake.", "label": 0},
    {"text": "New training facilities unveiled for the national hockey team.", "label": 0},
    {"text": "The underdog team surprises everyone with a stunning win.", "label": 0},
    {"text": "The swimming championship showcased extraordinary talent.", "label": 0},
    {"text": "The coach shares strategies ahead of the semi-finals.", "label": 0},
    {"text": "Athletes gear up for the international sports meet.", "label": 0},
    {"text": "The volleyball team secures a crucial win in the tournament.", "label": 0},
    {"text": "The government unveils a new healthcare policy.", "label": 1},
    {"text": "The president addresses the nation on economic reforms.", "label": 1},
    {"text": "The election results spark debates across the country.", "label": 1},
    {"text": "Parliament passes a bill to boost renewable energy.", "label": 1},
    {"text": "Diplomatic talks between the two nations yield progress.", "label": 1},
    {"text": "The mayor announces plans for urban development.", "label": 1},
    {"text": "The opposition criticizes the new tax policies.", "label": 1},
    {"text": "Campaigns heat up as candidates rally for voter support.", "label": 1},
    {"text": "The new law aims to strengthen cybersecurity measures.", "label": 1},
    {"text": "The prime minister visits a neighboring country for trade talks.", "label": 1},
    {"text": "Protests erupt against the controversial policy decision.", "label": 1},
    {"text": "A landmark decision by the court shapes future legislation.", "label": 1},
    {"text": "The minister highlights plans to improve education systems.", "label": 1},
    {"text": "National security is at the forefront of the political agenda.", "label": 1},
    {"text": "The governor outlines disaster relief efforts after the floods.", "label": 1},
    {"text": "A major treaty is signed to address climate change issues.", "label": 1},
    {"text": "Students prepare for university entrance exams.", "label": 2},
    {"text": "The new study techniques promise better learning outcomes.", "label": 2},
    {"text": "The education board announces changes in the curriculum.", "label": 2},
    {"text": "A groundbreaking study explores the effects of online learning.", "label": 2},
    {"text": "A new scholarship program supports underprivileged students.", "label": 2},
    {"text": "Teachers attend workshops on integrating technology in classrooms.", "label": 2},
    {"text": "The library adds hundreds of new academic resources.", "label": 2},
    {"text": "Researchers publish findings on improving memory retention.", "label": 2},
    {"text": "Study groups form to tackle challenging science topics.", "label": 2},
    {"text": "Students collaborate on innovative projects for the science fair.", "label": 2},
    {"text": "The school introduces coding classes for young learners.", "label": 2},
    {"text": "An education conference discusses the future of AI in learning.", "label": 2},
    {"text": "Experts share tips for mastering difficult math concepts.", "label": 2},
    {"text": "The university announces free courses on climate studies.", "label": 2},
    {"text": "A recent survey highlights the need for mental health support in schools.", "label": 2},
    {"text": "The tutoring center reports an increase in attendance.", "label": 2},
    {"text": "Students celebrate as their project wins the national innovation award.", "label": 2},
    {"text": "A new app helps students track their study habits effectively.", "label": 2},
]

# Split the data into training and validation sets
train_texts, val_texts = train_test_split(data, test_size=0.2, random_state=42)
train_dataset = Dataset.from_list(train_texts)
val_dataset = Dataset.from_list(val_texts)

# Tokenizer and Model
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set padding token as EOS

# Load the model without quantization initially
model = GPT2ForSequenceClassification.from_pretrained("distilgpt2", num_labels=3)
model.config.pad_token_id = tokenizer.pad_token_id

# LoRA Configuration
lora_config = LoraConfig(
    r=8,  # Rank of low-rank adapters
    lora_alpha=16,  # Scaling factor for LoRA
    target_modules=["attn.c_attn", "attn.c_proj", "mlp.c_fc", "mlp.c_proj"],  # Target modules
    task_type="SEQ_CLS",  # Task type for sequence classification
)

# Wrap Base Model with LoRA
model = get_peft_model(model, lora_config)

# Tokenization
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

# Remove unneeded columns
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_val = tokenized_val.remove_columns(["text"])

# Format to Torch
tokenized_train.set_format("torch")
tokenized_val.set_format("torch")

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results_qlora_finetune_gpt2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs_qlora_finetune_gpt2",
    logging_steps=10,
    no_cuda=True,  # Ensure training on CPU
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Apply dynamic quantization after fine-tuning
# Exclude LoRA layers from quantization
quantized_model = quantize_dynamic(
    model,
    {torch.nn.Linear},  # Only quantize Linear layers
    dtype=torch.qint8  # Quantize to 8-bit integer type
)

# Save the model's state_dict directly to avoid 'clone' error
torch.save(quantized_model.state_dict(), "./qlora_finetune_gpt2_model_quantized.pth")

# Save tokenizer separately
tokenizer.save_pretrained("./qlora_finetune_gpt2_model")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.439400,1.867554
2,1.392500,1.835588
3,1.388100,1.822397


('./qlora_finetune_gpt2_model/tokenizer_config.json',
 './qlora_finetune_gpt2_model/special_tokens_map.json',
 './qlora_finetune_gpt2_model/vocab.json',
 './qlora_finetune_gpt2_model/merges.txt',
 './qlora_finetune_gpt2_model/added_tokens.json')

In [3]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Evaluation function
def evaluate_model(trainer, tokenized_val):
    # Perform evaluation and get predictions
    eval_results = trainer.evaluate(tokenized_val)
    print("Evaluation Results:")
    print(f"Loss: {eval_results['eval_loss']:.4f}")

    # Get predictions and labels for accuracy and other metrics
    predictions = trainer.predict(tokenized_val)
    preds = np.argmax(predictions.predictions, axis=-1)
    labels = predictions.label_ids

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)
    print(f"Accuracy: {accuracy:.4f}")

    # Calculate Precision, Recall, F1 Score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return accuracy, precision, recall, f1

# Perform evaluation
accuracy, precision, recall, f1 = evaluate_model(trainer, tokenized_val)

# You can also save these metrics to a file if desired
metrics = {
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "f1": f1,
}

# Save metrics to a file
import json
with open("./evaluation_metrics.json", "w") as f:
    json.dump(metrics, f, indent=4)

print("Evaluation metrics saved to ./evaluation_metrics.json")


Evaluation Results:
Loss: 1.8224
Accuracy: 0.1000
Precision: 0.0100
Recall: 0.1000
F1 Score: 0.0182
Evaluation metrics saved to ./evaluation_metrics.json


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
